In [2]:
import pandas as pd
import numpy as np

# Indlæs filen
df_input = pd.read_csv('Data/input_landbrugsdata.csv')
df_timer = pd.read_csv('Data/Lontimer_landbrugsdata.csv')
#filter for løbende priser og foregående års priser
df_lonsum = df_input.loc[(df_input['TILGANG1'] == 'Aflønning af ansatte') & 
                         (df_input['PRISENHED'] == 'Løbende priser')]

#fjern prishenhed
df_lonsum.drop(columns=['PRISENHED'],inplace=True)
df_lonsum.drop(columns=['TILGANG1'],inplace=True)
df_timer.drop(columns=['SOCIO'],inplace=True)

#set index
df_lonsum.set_index(['ANVENDELSE', 'TID'],inplace=True)
df_timer.set_index(['BRANCHE', 'TID'],inplace=True)

# Rens 'ANVENDELSE' i df_lonsum ved kun at beholde de første 6 karakterer (koden)
df_lonsum.index = df_lonsum.index.set_levels(
    df_lonsum.index.levels[0].str.slice(0, 6), level=0
)

# Gør det samme for df_timer (her hedder niveauet 'BRANCHE' ifølge dit billede)
df_timer.index = df_timer.index.set_levels(
    df_timer.index.levels[0].str.slice(0, 6), level=0
)

#tjek
df_lonsum

C:\Users\b431385\AppData\Local\Temp\ipykernel_27468\460723647.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lonsum.drop(columns=['PRISENHED'],inplace=True)
C:\Users\b431385\AppData\Local\Temp\ipykernel_27468\460723647.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lonsum.drop(columns=['TILGANG1'],inplace=True)


INDHOLD
ANVENDELSE TID           
010000     2002  5285.780
100010     2002  6411.857
100030     2002  3222.280
010000     1998  4777.547
100010     1998  5862.680
...                   ...
100040     2018  8199.278
           2019  7981.833
           2020  8376.898
           2021  8285.645
           2022  8531.400

[285 rows x 1 columns]

In [3]:
# Sørg for at index-navnene er identiske, ellers kan Pandas brokke sig ved division
df_timer.index.names = ['ANVENDELSE', 'TID']
df_timer

INDHOLD
ANVENDELSE TID          
010000     1966   630077
100010     1966    60113
100030     1966    30437
010000     1967   595974
100010     1967    54768
...                  ...
100040     2018    29900
           2019    29035
           2020    29401
           2021    29055
           2022    28888

[285 rows x 1 columns]

In [4]:
# 1. Udfør divisionen specifikt på 'INDHOLD' kolonnerne
# Dette sikrer, at vi kun dividerer tallene
df_timelon = df_lonsum['INDHOLD'] / (df_timer['INDHOLD'] / 1000)

# 2. Gør det til et DataFrame og giv kolonnen et sigende navn
df_timelon = df_timelon.to_frame(name='TIMELOEN_KR')

# 3. FLYT INDEX TIL KOLONNER (Dette er løsningen på dit problem!)
# Når du gør dette, bliver 'ANVENDELSE' og 'TID' til rigtige kolonner i din CSV
df_output = df_timelon.reset_index()

# 4. Gem til CSV
df_output.to_csv('Data/TimeLon_landbrugsdata.csv', index=False)

# Vis resultatet for at tjekke
df_output

,ANVENDELSE,TID,TIMELOEN_KR
0,010000,1966,1.364024
1,010000,1967,1.459260
2,010000,1968,1.537552
3,010000,1969,1.635656
4,010000,1970,1.587802
...,...,...,...
280,REST_A,2018,285.982425
281,REST_A,2019,292.818139
282,REST_A,2020,307.817315
283,REST_A,2021,306.633561
